In [1]:
import numpy as np
import pandas as pd
import itertools
import collections
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
dataFrame = pd.read_csv('Attachment/data.csv')
dataFrame.head()

,index,authors,category,date,headline,link,short_description
0,0,"Katherine LaGrave, ContributorTravel writer an...",TRAVEL,2014-05-07,"EccentriCities: Bingo Parties, Paella and Isla...",https://www.huffingtonpost.com/entry/eccentric...,Påskekrim is merely the tip of the proverbial ...
1,1,Ben Hallman,BUSINESS,2014-06-09,Lawyers Are Now The Driving Force Behind Mortg...,https://www.huffingtonpost.com/entry/mortgage-...,NaN
2,2,Jessica Misener,STYLE & BEAUTY,2012-03-12,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,https://www.huffingtonpost.com/entry/madonna-s...,"Madonna is slinking her way into footwear now,..."
3,3,"Victor and Mary, Contributor\n2Sense-LA.com",TRAVEL,2013-12-17,Sophistication and Serenity on the Las Vegas S...,https://www.huffingtonpost.com/entry/las-vegas...,But what if you're a 30-something couple that ...
4,4,"Emily Cohn, Contributor",BUSINESS,2015-03-19,It's Still Pretty Hard For Women To Get Free B...,https://www.huffingtonpost.com/entry/free-birt...,Obamacare was supposed to make birth control f...


In [3]:
data = dataFrame.short_description
data = data.fillna('')
data = data.str.lower()
data = data.str.replace('[^\w\s]',' ')
data = data.str.replace('\d+', '')
stopWords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
wordsList = data.apply(word_tokenize)
wordsList = wordsList.apply(lambda x: [item for item in x if item not in stopWords])
wordsList = wordsList.apply(lambda x: [lemmatizer.lemmatize(y, pos="v") for y in x])
dataFrame['words'] = wordsList
dataFrame.head()

,index,authors,category,date,headline,link,short_description,words
0,0,"Katherine LaGrave, ContributorTravel writer an...",TRAVEL,2014-05-07,"EccentriCities: Bingo Parties, Paella and Isla...",https://www.huffingtonpost.com/entry/eccentric...,Påskekrim is merely the tip of the proverbial ...,"[påskekrim, merely, tip, proverbial, iceberg, ..."
1,1,Ben Hallman,BUSINESS,2014-06-09,Lawyers Are Now The Driving Force Behind Mortg...,https://www.huffingtonpost.com/entry/mortgage-...,NaN,[]
2,2,Jessica Misener,STYLE & BEAUTY,2012-03-12,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,https://www.huffingtonpost.com/entry/madonna-s...,"Madonna is slinking her way into footwear now,...","[madonna, slink, way, footwear, truth, dare, p..."
3,3,"Victor and Mary, Contributor\n2Sense-LA.com",TRAVEL,2013-12-17,Sophistication and Serenity on the Las Vegas S...,https://www.huffingtonpost.com/entry/las-vegas...,But what if you're a 30-something couple that ...,"[something, couple, shy, away, table, dance, e..."
4,4,"Emily Cohn, Contributor",BUSINESS,2015-03-19,It's Still Pretty Hard For Women To Get Free B...,https://www.huffingtonpost.com/entry/free-birt...,Obamacare was supposed to make birth control f...,"[obamacare, suppose, make, birth, control, fre..."


In [4]:
travel = dataFrame[dataFrame['category'] == 'TRAVEL']
business = dataFrame[dataFrame['category'] == 'BUSINESS']
sb = dataFrame[dataFrame['category'] == 'STYLE & BEAUTY']
trainTravel = travel.sample(frac=0.8)
evaluateTravel = travel.drop(trainTravel.index)
trainBusiness = business.sample(frac=0.8)
evaluateBusiness = business.drop(trainBusiness.index)
trainSB = sb.sample(frac=0.8)
evaluateSB = sb.drop(trainSB.index)
trainData = pd.concat([trainTravel,trainBusiness, trainSB])
trainData = trainData.sample(frac=1)
evaluateData = pd.concat([evaluateSB,evaluateBusiness, evaluateTravel])
evaluateData = evaluateData.sample(frac=1)
trainData.head()

,index,authors,category,date,headline,link,short_description,words
1723,1723,NaN,TRAVEL,2013-11-25,A Pond So Beautiful It Belongs On Your Desktop...,https://www.huffingtonpost.com/entry/blue-pond...,We’re not kidding. Check your Macbook wallpape...,"[kid, check, macbook, wallpaper, pond, mood, h..."
11993,11993,NaN,STYLE & BEAUTY,2012-07-08,Sienna Miller Reportedly Gives Birth,https://www.huffingtonpost.com/entry/sienna-mi...,"Details on the baby's gender, name and weight ...","[detail, baby, gender, name, weight, yet, repo..."
21058,21058,Simone Kitchens,STYLE & BEAUTY,2012-09-20,"Beauty Street Style: Julia Panek, Graphic Desi...",https://www.huffingtonpost.com/entry/beauty-st...,"We've long been a fan of Joan Holloway, Christ...","[long, fan, joan, holloway, christina, hendric..."
19785,19785,"Greg Portell , ContributorPartner, A.T. Ke...",BUSINESS,2015-05-28,Marketers are Prepping for FIFA Fallout,https://www.huffingtonpost.com/entry/marketers...,While the extent of the criminal charges are e...,"[extent, criminal, charge, eye, open, fact, fi..."
12349,12349,"Rick Steves, Contributor\nContributor",TRAVEL,2013-11-18,"The Security Fence, the Anti-Terrorism Barrier...",https://www.huffingtonpost.com/entry/the-secur...,"Since 2003, Israel has been building a wall ar...","[since, israel, build, wall, around, west, ban..."


In [5]:
travelWords = trainTravel.words.values
travelWords = list(itertools.chain.from_iterable(travelWords))
BusinessWords = trainBusiness.words.values
BusinessWords = list(itertools.chain.from_iterable(BusinessWords))
allTravelAndBusinessWord = list(map(''.join, set(itertools.chain(travelWords, BusinessWords))))
print(len(travelWords),len(BusinessWords),len(allTravelAndBusinessWord))
newDataFrame = pd.DataFrame(columns=['Word','Travel occurrences','Business occurrences'])
for word in allTravelAndBusinessWord:
    newDataFrame = newDataFrame.append({'Word' : word,'Travel occurrences' : travelWords.count(word),'Business occurrences' : BusinessWords.count(word)},ignore_index=True)


103878 51724 17831


In [6]:
newDataFrame['Travel Probability'] = newDataFrame['Travel occurrences'] / newDataFrame['Travel occurrences'].sum()
newDataFrame['Business Probability'] = newDataFrame['Business occurrences'] / newDataFrame['Business occurrences'].sum()
newDataFrame = newDataFrame.set_index('Word')
newDataFrame

,Travel occurrences,Business occurrences,Travel Probability,Business Probability
Word,,,,
guam,1,0,9.62668e-06,0
new,547,301,0.00526579,0.00581935
thirds,0,2,0,3.86668e-05
casita,2,0,1.92534e-05,0
rove,4,0,3.85067e-05,0
...,...,...,...,...
implement,1,12,9.62668e-06,0.000232001
momentarily,1,0,9.62668e-06,0
glitteringly,1,0,9.62668e-06,0


In [7]:
# travelPriorProbability = len(trainTravel)/(len(trainTravel)+len(trainBusiness))
# businessPriorProbability = len(trainBusiness)/(len(trainTravel)+len(trainBusiness))
# print(travelPriorProbability,businessPriorProbability)

In [8]:
travelAndBusinessEvaluateData = pd.concat([evaluateBusiness, evaluateTravel])
travelAndBusinessEvaluateData = travelAndBusinessEvaluateData.sample(frac=1)


In [9]:
for index,row in travelAndBusinessEvaluateData.iterrows():
    travelPriorProbability = len(trainTravel)/(len(trainTravel)+len(trainBusiness))
    businessPriorProbability = len(trainBusiness)/(len(trainTravel)+len(trainBusiness))
    words = set(row['words'])
    for word in words:
        if word in allTravelAndBusinessWord:
            travelPriorProbability *= newDataFrame.at[word,'Travel Probability']
            businessPriorProbability *= newDataFrame.at[word,'Business Probability']

    travelAndBusinessEvaluateData.at[index,'Travel Probability'] = travelPriorProbability
    travelAndBusinessEvaluateData.at[index,'Business Probability'] = businessPriorProbability
    if travelPriorProbability >= businessPriorProbability:
        travelAndBusinessEvaluateData.at[index,'Prediction'] = 'TRAVEL'
    else:
        travelAndBusinessEvaluateData.at[index,'Prediction'] = 'BUSINESS'
travelAndBusinessEvaluateData

,index,authors,category,date,headline,link,short_description,words,Travel Probability,Business Probability,Prediction
19148,19148,"Bob Schulman, Contributor\nTravel Editor, Watc...",TRAVEL,2012-10-06,A Paradise For Copper Lovers In Mexico,https://www.huffingtonpost.com/entry/a-paradis...,"If you like things made out of copper, you'll ...","[like, things, make, copper, love, little, vil...",4.040528e-107,0.000000e+00,TRAVEL
234,234,"Lonely Planet, Contributor\nLonely Planet is t...",TRAVEL,2012-02-01,A Love Surprise: Perfect Valentine's Day Getaw...,https://www.huffingtonpost.com/entry/a-love-su...,Why settle for a card and flowers for this Val...,"[settle, card, flower, valentine, day, travel,...",5.084963e-21,0.000000e+00,TRAVEL
13222,13222,"Alex Ivanovs, ContributorWriter",BUSINESS,2014-10-02,5 Types of Web Content for Driving Extra Traffic,https://www.huffingtonpost.com/entry/5-types-o...,Trying out something new shouldn't be feared -...,"[try, something, new, fear, feel, right, perha...",1.394341e-48,3.523846e-47,BUSINESS
1369,1369,"Quora, Contributor\nThe best answer to any que...",TRAVEL,2013-03-15,"So What Is This ""St.Patrick's Day"" Nonsense Al...",https://www.huffingtonpost.com/entry/so-what-i...,[2] Saint I've been in a couple of discussions...,"[saint, couple, discussions, late, around, dan...",5.063032e-34,8.578930e-35,TRAVEL
15961,15961,"George Hobica, Contributor\nSyndicated travel ...",TRAVEL,2012-08-22,Ask Airfarewatchdog: JetBlue Schedule Change A...,https://www.huffingtonpost.com/entry/jetblue-s...,"Come on JetBlue, step up.","[come, jetblue, step]",8.445674e-11,0.000000e+00,TRAVEL
...,...,...,...,...,...,...,...,...,...,...,...
7795,7795,Shane Ferro,BUSINESS,2015-09-11,New Study Finds Recessions Mean Fewer Babies,https://www.huffingtonpost.com/entry/recession...,The effects appear to linger even after the ec...,"[effect, appear, linger, even, economy, back, ...",6.914080e-25,5.702471e-25,TRAVEL
21989,21989,NaN,TRAVEL,2012-10-25,Morgue Hotel? Haydn Pearce Plots To Turn Slabs...,https://www.huffingtonpost.com/entry/morgue-ho...,"Haydn Pearce, a Tasmanian businessman, is hopi...","[haydn, pearce, tasmanian, businessman, hop, t...",4.222783e-27,0.000000e+00,TRAVEL
1060,1060,NaN,BUSINESS,2015-12-14,How AT&T Execs Took Over The Red Cross And Hur...,https://www.huffingtonpost.comhttps://www.prop...,WHEN GAIL MCGOVERN WAS PICKED to head the Amer...,"[gail, mcgovern, pick, head, american, red, cr...",0.000000e+00,0.000000e+00,TRAVEL
8352,8352,Dave Jamieson,BUSINESS,2017-01-12,"Amazon Says It Plans To Add 100,000 Jobs",https://www.huffingtonpost.com/entry/amazon-ad...,Most of those positions will probably be in th...,"[position, probably, retailer, grow, number, f...",3.197045e-28,4.563870e-25,BUSINESS


In [10]:
travelAndBusinessEvaluateData['correct'] = (travelAndBusinessEvaluateData['category'] == travelAndBusinessEvaluateData['Prediction'])
correctDetected = (travelAndBusinessEvaluateData['correct']).sum()
accuracy =  correctDetected / len(travelAndBusinessEvaluateData)
accuracy

0.7883467883467884